In [13]:
import orjson
import numpy as np
import math

In [14]:
with open("../resources/data_final.json", "rb") as f:
    dataset = orjson.loads(f.read())

In [24]:
for i in dataset:
    print(i)

{'context': 'capitulo ix orientacoes pesquisa extensao cursos degraduacao uaeadtecart processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio plataformas tecnologias digitais bemcomo auxilio ferramentas ava unidade academica educacao distancia etecnologia uaeadtecart docentes orientadores projetos pesquisa pibicpic eoutros ensino eou extensao sonus bext poderao criar salas virtuais orientacoes noava uaeadtec visando registro atividades orientacao apoiar discentes nasacoes propostas', 'qas': [{'id': '118030826', 'is_impossible': False, 'question': 'Quando começam as aulas do PLE?', 'answers': [{'text': 'O primeiro PLE (2020.3) iniciará no dia 17 de agosto. Confira o Calendário Acadêmico com todas as datas ', 'answer_start': 0}]}]}
{'context': 'capitulo ix orientacoes pesquisa extensao cursos degraduacao uaeadtecart processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio plataformas tecnologias digitais bemcomo auxilio ferrament

In [15]:
np.random.seed(0)
id_train = np.random.choice(len(dataset), math.ceil(len(dataset)/2))

In [16]:
id_train.shape

(48,)

In [17]:
id_test = np.asarray(list(set(range(len(dataset))).difference(set(id_train))))

In [18]:
id_test.shape

(60,)

In [19]:
# https://github.com/ThilinaRajapakse/simpletransformers#question-answering

In [20]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create dummy data to use for training.
train_data = [dataset[id] for id in id_train]
print(train_data)

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 'distilbert-base-uncased-distilled-squad', args={'reprocess_input_data': True, 'overwrite_output_dir': True})

# Train the model with JSON file
# model.train_model('data/train.json')

# The list can also be used directly
model.train_model(train_data)

# Evaluate the model. (Being lazy and evaluating on the train data itself)
result, text = model.eval_model('data/train.json')

print(result)
print(text)

print('-------------------')

# Making predictions using the model.
to_predict = np.asarray([dataset[id] for id in id_test])


print(model.predict(to_predict))

[{'context': 'capitulo ix orientacoes pesquisa extensao cursos degraduacao uaeadtecart processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio plataformas tecnologias digitais bemcomo auxilio ferramentas ava unidade academica educacao distancia etecnologia uaeadtecart docentes orientadores projetos pesquisa pibicpic eoutros ensino eou extensao sonus bext poderao criar salas virtuais orientacoes noava uaeadtec visando registro atividades orientacao apoiar discentes nasacoes propostas', 'qas': [{'id': '118030870', 'is_impossible': False, 'question': 'Como funciona a equiparação? Todos os cursos, incluindo licenciaturas, podem ter?', 'answers': [{'text': 'A equiparação funcionará de acordo com as regras constantes no PPC de cada curso e em acordo com a resolução 425/2010\xa0 CEPE/UFRPE. A equiparação de estágio pode ser feita, se prevista no PPC do curso. Caso não seja prevista, o CCD e NDE do curso podem de forma ágil possibilitar a equiparação, já para o PL

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
Could not find answer: 'capitulo ix orientacoes pesquisa extensao cursos degraduacao uaeadtecart processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio plataformas tecnologias digitais bemcomo auxilio ferramentas ava unidade academica educacao distancia etecnologia uaeadtecart docentes orientadores projetos' vs. 'A equiparação funcionará de acordo com as regras constantes no PPC de cada curso e em acordo com a resolução 425/2010 CEPE/UFRPE. A equiparação de estágio pode ser feita, se prevista no PPC do curso. Caso não seja prevista, o CCD e NDE do curso podem de forma ágil possibilitar a equiparação, já para o PLE.'
convert squad examples to features:   0%|          | 0/48 [00:00<?, ?it/s]egraduacao uaeadtecart processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio plataformas tecnologias digitais bemcomo auxilio ferramentas 

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [22]:
from transformers import pipeline
qapipe = pipeline('question-answering')
qapipe({
    'question': """Como funciona a equiparação? Todos os cursos, incluindo licenciaturas, podem ter?""",
    'context': """capitulo ix orientacoes pesquisa extensao cursos degraduacao uaeadtecart
    processos orientacao atividades pesquisa eextensao serao realizados modo remoto apoio
    plataformas tecnologias digitais bemcomo auxilio ferramentas ava unidade academica
    educacao distancia etecnologia uaeadtecart docentes orientadores projetos pesquisa
    pibicpic eoutros ensino eou extensao sonus bext poderao criar salas virtuais orientacoes
    noava uaeadtec visando registro atividades orientacao apoiar discentes nasacoes propostas"""
})

/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.0009449739591218531,
 'start': 500,
 'end': 527,
 'answer': 'discentes nasacoes propostas'}